In [105]:
'''Usage: python cal_processing.py idx start_date end_date observations_path simulation_path output_path'''

import sys
import numpy as np
import pandas as pd

# define the objective functions from Bart et al. 2016

#NSE
def NSE(obs,mod):
       
    return 1. - (np.nansum(np.square(obs-mod)) / np.nansum(np.square(obs - np.nanmean(obs))))

                 
def NSE_log(obs,mod):
    
    obs = np.log10(obs)
    
    mod[mod<=0] = np.NaN
    
    mod = np.log10(mod)
    
    return 1. - (np.nansum(np.square(obs-mod)) / np.nansum(np.square(obs - np.nanmean(obs))))

def PE(obs,mod):
    
    return (np.nanmean(mod)-np.nanmean(obs))/np.nanmean(obs)

def OBFX(obs,mod):
    
    pe = PE(obs,mod)
    nse = NSE(obs,mod)
    nse_log = NSE_log(obs,mod)
    
    obfx = nse*nse_log*(1.-np.abs(pe))
    
    return pe,nse,nse_log

def parse_rhessys_date(df):
    return '%s-%s-%s'%(int(df.year),int(df.month),int(df.day))


# parse the command line arguments
idx = sys.argv[1]
strt = sys.argv[2]
nd = sys.argv[3]
obspth = sys.argv[4]
modpth = sys.argv[5]
outpth = sys.argv[6]

#idx = '01'
#strt = '2004-10-01'
#nd = '2012-09-29'
#obspth = './data/como_q_obs.pcl'
#modpth = '/RHESSys/Como/out/test_basin.daily'
#outpth = '/RHESSys/Como/out/test_streamflow.npz'

In [106]:
# read in the observations
obs = pd.read_pickle(obspth)[strt:nd].discharge.as_matrix()

In [107]:
# read in the model
mod = pd.read_table(modpth,sep=' ')
mod['datetime'] = mod.apply(parse_rhessys_date,axis = 1)
mod.index = pd.DatetimeIndex(mod.datetime)
mod = mod[strt:nd].streamflow.as_matrix()

np.savez_compressed(outpth,streamflow=mod)

In [108]:
pe,nse,nse_log = OBFX(obs,mod)

In [109]:
print '%s,%s,%s,%s'%(idx,pe,nse,nse_log)

01,-0.620480581439,-2.20827515398,0.915246227918
